In [1]:
%cd ..

c:\Users\HuyLQ15_CTV\Desktop\model_monitoring


In [2]:
from sklearn.datasets import load_iris
from sklearn.utils import shuffle

data = load_iris(as_frame=True)['data']
data = shuffle(data, random_state=42)
data['target'] = load_iris(as_frame=True)['target']

In [3]:
from sklearn.linear_model import LogisticRegression

X = data.drop('target', axis=1).iloc[:125]
y = data['target'].iloc[:125]
model = LogisticRegression()
model.fit(X, y)
prediction1 = model.predict(data.drop('target', axis=1))
prediction2 = model.predict_proba(data.drop('target', axis=1))
data['prediction'] = prediction1
data[['0','1','2']] = prediction2

In [4]:
import pandas as pd
data2 = pd.DataFrame(prediction2, columns=model.classes_)
data2['target'] = data['target']

In [5]:
reference = data.iloc[:75]
current = data.iloc[75:]

In [6]:
labels=['0','1','2']

# Kế hoạch
Có 2 option để truyền vào column mapping
- Kết quả dạng đã chuyển thành predict label
- Kết quả dạng xác suất


## Transformed labels

### Utils

In [7]:
from evidently.base_metric import MetricResult, InputData

from typing import Optional
class MultiClassificationResult(MetricResult):
    class Config:
        type_alias = "evidently:metric_result:MultiClassificationResult"
    accuracy: float
    confusion_matrix: dict
    classification_info: dict
    logloss: Optional[float]
    roc_auc: Optional[dict]
    mattheus_corr_coefficient: float

In [8]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss

from source.calculator.classification_performance import calculate_confusion_by_classes 
from sklearn.metrics import confusion_matrix
def calculate_confusion_matrix_by_classes(y_true, y_pred):
    labels = list(set(y_true) | set(y_pred))
    confusion_matrix_inf = confusion_matrix(y_true, y_pred, labels=labels)
    return calculate_confusion_by_classes(confusion_matrix_inf, class_names=labels)

### Metrics

In [9]:
def calculate_multi_classification(
    prediction: pd.Series, 
    target: pd.Series,
    prediction_proba: pd.DataFrame = None
    ) -> MultiClassificationResult:

    prediction = prediction.map(str)
    target = target.map(str)

    logloss = None
    roc_auc = None
    
    accuracy = accuracy_score(target, prediction)
    classification_info = classification_report(target, prediction, output_dict=True)
    classification_info.pop('accuracy')
    confusion_matrix = calculate_confusion_matrix_by_classes(target, prediction)
    mattheus_corr_coefficient = matthews_corrcoef(target, prediction)
    if prediction_proba is not None:
        try:
            roc_auc = {}
            roc_auc['ovr'] = float(roc_auc_score(y_true=prediction, y_score=prediction_proba, multi_class='ovr'))
            roc_auc['ovo'] = float(roc_auc_score(y_true=prediction, y_score=prediction_proba, multi_class='ovo'))
            logloss = log_loss(y_true=prediction, y_pred=prediction_proba)
        except:
            pass
    return MultiClassificationResult(
        accuracy = accuracy,
        confusion_matrix = confusion_matrix,
        classification_info = classification_info,
        logloss = logloss,
        roc_auc = roc_auc,
        mattheus_corr_coefficient = float(mattheus_corr_coefficient)
    )

In [10]:
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.utils.data_operations import process_columns
from evidently.calculations.classification_performance import get_prediction_data

column_mapping = ColumnMapping()
column_mapping.prediction = ['0','1','2']
column_mapping.target = 'target'


dataset_column = process_columns(data, column_mapping)
prediction_data = get_prediction_data(data, data_columns=dataset_column, pos_label=None, threshold = 0.5)

results = calculate_multi_classification(
    prediction=prediction_data.predictions,
    target=data['target'],
    prediction_proba=prediction_data.prediction_probas
)
results

MultiClassificationResult(type='evidently:metric_result:MultiClassificationResult', accuracy=0.96, confusion_matrix={'0': {'tp': 50.0, 'tn': 100.0, 'fp': 0.0, 'fn': 0.0}, '1': {'tp': 48.0, 'tn': 96.0, 'fp': 4.0, 'fn': 2.0}, '2': {'tp': 46.0, 'tn': 98.0, 'fp': 2.0, 'fn': 4.0}}, classification_info={'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 50.0}, '1': {'precision': 0.9230769230769231, 'recall': 0.96, 'f1-score': 0.9411764705882353, 'support': 50.0}, '2': {'precision': 0.9583333333333334, 'recall': 0.92, 'f1-score': 0.9387755102040817, 'support': 50.0}, 'macro avg': {'precision': 0.9604700854700855, 'recall': 0.96, 'f1-score': 0.9599839935974389, 'support': 150.0}, 'weighted avg': {'precision': 0.9604700854700855, 'recall': 0.96, 'f1-score': 0.9599839935974391, 'support': 150.0}}, logloss=0.12454405235626946, roc_auc={'ovr': 1.0, 'ovo': 1.0}, mattheus_corr_coefficient=0.9402507669779171)

In [11]:
from evidently.base_metric import Metric
class MultiClassificationPerformanceResult(MetricResult):
    class Config:
        type_alias = "evidently:metric_result:MultiClassificationPerformanceResult"
    reference: Optional[MultiClassificationResult]
    current: MultiClassificationResult

class MultiClassificationPerformanceMetric(Metric[MultiClassificationPerformanceResult]):
    class Config:
        type_alias = "evidently:metric:MultiClassificationPerformanceMetric"
    def __init__(self):
        super().__init__()

    def calculate(self, data: InputData) -> MultiClassificationPerformanceResult:
        results = {}
        results['reference'] = None
        if data.current_data is None:
            raise ValueError("The value cannot be None")

        if data.reference_data is not None:
            results['reference'] = self.get_classification_result(data.reference_data, data.column_mapping)
            
        results['current'] = self.get_classification_result(data.current_data, data.column_mapping)
        return MultiClassificationPerformanceResult(
            reference=results['reference'],
            current=results['current']
        )
        
    def get_classification_result(self, data: pd.DataFrame, column_mapping: ColumnMapping) -> MultiClassificationResult:
        dataset_column = process_columns(data, column_mapping)
        prediction_data = get_prediction_data(data, data_columns=dataset_column, pos_label=None, threshold = 0.5)
        results = calculate_multi_classification(
            prediction=prediction_data.predictions,
            target=data[dataset_column.utility_columns.target],
            prediction_proba=prediction_data.prediction_probas
        )
        return results

In [12]:
from evidently.renderers.base_renderer import MetricRenderer, default_renderer

@default_renderer(wrap_type=MultiClassificationPerformanceMetric)
class MultiClassificationPerformanceRender(MetricRenderer):
    def render_json(self, obj: MultiClassificationPerformanceMetric, include_render: bool = False,
        include: "IncludeOptions" = None, exclude: "IncludeOptions" = None,) -> dict:
        result = obj.get_result().get_dict(include_render, include, exclude)
        return result

In [13]:
from evidently.report import Report

report = Report(metrics=[
    MultiClassificationPerformanceMetric()
])

report.run(current_data=reference, reference_data=reference, column_mapping=column_mapping)
report.json()
# report.save_json('./reports/custom/Multi Classification Performance.json')

'{"version": "0.4.39", "metrics": [{"metric": "MultiClassificationPerformanceMetric", "result": {"reference": {"accuracy": 0.9733333333333334, "confusion_matrix": {"0": {"tp": 29.0, "tn": 46.0, "fp": 0.0, "fn": 0.0}, "1": {"tp": 23.0, "tn": 50.0, "fp": 2.0, "fn": 0.0}, "2": {"tp": 21.0, "tn": 52.0, "fp": 0.0, "fn": 2.0}}, "classification_info": {"0": {"precision": 1.0, "recall": 1.0, "f1-score": 1.0, "support": 29.0}, "1": {"precision": 0.92, "recall": 1.0, "f1-score": 0.9583333333333334, "support": 23.0}, "2": {"precision": 1.0, "recall": 0.9130434782608695, "f1-score": 0.9545454545454546, "support": 23.0}, "macro avg": {"precision": 0.9733333333333333, "recall": 0.9710144927536232, "f1-score": 0.970959595959596, "support": 75.0}, "weighted avg": {"precision": 0.9754666666666666, "recall": 0.9733333333333334, "f1-score": 0.9732828282828283, "support": 75.0}}, "logloss": 0.11196622485325247, "roc_auc": {"ovr": 1.0, "ovo": 1.0}, "mattheus_corr_coefficient": 0.9607743323674646}, "current

In [14]:
from source.metrics import MultiClassificationPerformanceMetric
report = Report(metrics=[
    MultiClassificationPerformanceMetric()
])

report.run(current_data=current, reference_data=reference, column_mapping=column_mapping)
report.json()

c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\evidently\pydantic_utils.py:161: UserWarning: Duplicate key (<class 'evidently.base_metric.Metric'>, 'evidently:metric:RegressionPerformanceMetrics') in alias map
  warnings.warn(f"Duplicate key {key} in alias map")
c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\evidently\pydantic_utils.py:161: UserWarning: Duplicate key (<class 'evidently.base_metric.MetricResult'>, 'evidently:metric_result:MultiClassificationResult') in alias map
  warnings.warn(f"Duplicate key {key} in alias map")
c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\evidently\pydantic_utils.py:161: UserWarning: Duplicate key (<class 'evidently.base_metric.MetricResult'>, 'evidently:metric_result:MultiClassificationPerformanceResult') in alias map
  warnings.warn(f"Duplicate key {key} in alias map")
c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monito

'{"version": "0.4.39", "metrics": [{"metric": "MultiClassificationPerformanceMetric", "result": {"reference": {"accuracy": 0.9733333333333334, "confusion_matrix": {"0": {"tp": 29.0, "tn": 46.0, "fp": 0.0, "fn": 0.0}, "1": {"tp": 23.0, "tn": 50.0, "fp": 2.0, "fn": 0.0}, "2": {"tp": 21.0, "tn": 52.0, "fp": 0.0, "fn": 2.0}}, "classification_info": {"0": {"precision": 1.0, "recall": 1.0, "f1-score": 1.0, "support": 29.0}, "1": {"precision": 0.92, "recall": 1.0, "f1-score": 0.9583333333333334, "support": 23.0}, "2": {"precision": 1.0, "recall": 0.9130434782608695, "f1-score": 0.9545454545454546, "support": 23.0}, "macro avg": {"precision": 0.9733333333333333, "recall": 0.9710144927536232, "f1-score": 0.970959595959596, "support": 75.0}, "weighted avg": {"precision": 0.9754666666666666, "recall": 0.9733333333333334, "f1-score": 0.9732828282828283, "support": 75.0}}, "logloss": 0.11215725602203173, "roc_auc": {"ovr": 1.0, "ovo": 1.0}, "mattheus_corr_coefficient": 0.9607743323674646}, "current

In [15]:
from evidently.metrics import ConflictPredictionMetric
from evidently.metrics import ClassificationQualityMetric
from evidently.metrics import ClassificationClassBalance
from evidently.metrics import ClassificationConfusionMatrix
from evidently.metrics import ClassificationQualityByClass
from evidently.metrics import ClassificationClassSeparationPlot
from evidently.metrics import ClassificationProbDistribution
from evidently.metrics import ClassificationRocCurve
from evidently.metrics import ClassificationPRCurve
from evidently.metrics import ClassificationPRTable
from evidently.metrics import ClassificationLiftCurve
from evidently.report import Report

classification_report = Report(metrics=[
    ClassificationQualityMetric(),
    ClassificationClassBalance(),
    ClassificationConfusionMatrix(),
    ClassificationQualityByClass(),
    # ClassificationRocCurve(),
    # ClassificationPRCurve(),
    # ClassificationLiftCurve(),
])

classification_report.run(reference_data=reference, current_data=current)
classification_report.save_html("./reports/default/classification_report.html")

In [16]:
from source.metric_preset.multi_classification import MultiClassificationPreset

classification_report = Report(metrics=[
    MultiClassificationPreset()
])

classification_report.run(reference_data=reference, current_data=current)
classification_report.save_html("./reports/custom/Classification Report.html")

c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\evidently\pydantic_utils.py:161: UserWarning:

Duplicate key (<class 'evidently.metric_preset.metric_preset.MetricPreset'>, 'evidently:metric_preset:DataDriftPreset') in alias map

c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\evidently\pydantic_utils.py:161: UserWarning:

Duplicate key (<class 'evidently.metric_preset.metric_preset.MetricPreset'>, 'evidently:metric_preset:DataQualityPreset') in alias map

